In [35]:
import kdl_parser_py.urdf
import PyKDL as kdl
import numpy as np
ok, tree = kdl_parser_py.urdf.treeFromFile("./urdf/stretch_iktut.urdf")
chain = tree.getChain("base_link", "link_grasp_center")
# ikvelsolver = kdl.ChainIkSolverVel_pinv(chain)
print(f"joints: {chain.getNrOfJoints()}, links: {chain.getNrOfSegments()}")

The root link base_link has an inertia specified in the URDF, but KDL does not support a root link with an inertia.  As a workaround, you can add an extra dummy link to your URDF.
joints: 7, links: 11


In [57]:
# q_dot = kdl.JntArray(6)
q = kdl.JntArray(7)
for i, j in enumerate([0,1,0,0,0,1,1]):
    q[i] = j
jac_solver = kdl.ChainJntToJacSolver(chain)
# t = kdl.Twist(
#     kdl.Vector(1,1,1),
#     kdl.Vector(0,0,0)
# )
# ikvelsolver.CartToJnt(q, t, q_dot)
# q_dot
J = kdl.Jacobian(7)
jac_solver.JntToJac(q,J)
print(J)
J = np.array([[J[i,j] for j in range(J.columns())] for i in range(J.rows())])
J_pinv = np.linalg.pinv(J)
with np.printoptions(precision=4, suppress=True):
    print(J)
    print(J_pinv)

           1 -3.67321e-06  2.77426e-11  2.77426e-11  2.77426e-11  2.77388e-11     0.111019
           0 -3.67322e-06           -1           -1           -1           -1     0.172904
           0            1 -7.34642e-06 -7.34642e-06 -7.34642e-06 -7.34642e-06  2.31314e-06
           0            0            0            0            0            0 -3.67321e-06
           0            0            0            0            0            0 -1.10196e-05
           0            0            0            0            0            0            1
[[ 1.     -0.      0.      0.      0.      0.      0.111 ]
 [ 0.     -0.     -1.     -1.     -1.     -1.      0.1729]
 [ 0.      1.     -0.     -0.     -0.     -0.      0.    ]
 [ 0.      0.      0.      0.      0.      0.     -0.    ]
 [ 0.      0.      0.      0.      0.      0.     -0.    ]
 [ 0.      0.      0.      0.      0.      0.      1.    ]]
[[ 1.      0.      0.      0.      0.     -0.111 ]
 [ 0.     -0.      1.      0.      0.     -0.   